In [1]:
# !pip install pandas
# !pip install ollama
#!pip install 'diffusers[torch]' transformers
#!pip install TTS
#!pip install moviepy
#!pip install opencv-python

In [2]:
!export DISABLE_TELEMETRY=YES

In [3]:
import ollama
import pandas as pd
import json

In [4]:
df = pd.read_csv('approved_stories.csv')

In [5]:
df

,Unnamed: 0.1,Unnamed: 0,title,selftext,ups,approval
0,0,74233,TIFU by having my reddit history revealed by J...,This happened to me yesterday and I'm still ba...,85249,True
1,3,35184,TIFU by getting Reddit banned in Russia,"Today Reddit was blocked in Russia, and I am t...",64262,True
2,4,73236,TIFU by not telling my wife our son was coming...,This happened Sunday night.\n\n\n\nMy oldest s...,49877,True
3,5,41777,TIFU by sitting in the wrong class for an enti...,I'm in my third year of University taking engi...,48085,True
4,6,77396,TIFU by trying to play my dad’s didgeridoo.,So my dad has had this didgeridoo in the house...,46906,True
...,...,...,...,...,...,...
7328,9994,76707,TIFU by Getting a Sodastream for my Parents fo...,This happened about 10 minutes ago. So several...,130,True
7329,9995,42961,TIFUpdate by making a dumb joke to myself whic...,[Some of you may remember my stupid joke](http...,130,True
7330,9996,71249,TIFU by pressing the wrong button,"So by some stroke of luck, I was invited to at...",130,True
7331,9997,73043,TIFU my first edible experience,"Mandatory ""This actually happened about a year...",130,True


In [6]:
from random import randint
storynum = randint(1, 7000)

In [7]:
inspo = df.iloc[storynum].selftext



print(inspo)

I'd rather orally pleasure a cactus than go through the traumatic event was a dinner 'party' a second time around. 

So I'm dating an American girl who's family is Moroccan. I swear on all that's holy and good in this world that her name is actually Jennifer, but with this spamming of her name, we can call her Penny. Moroccan family's will occasionally make a dish and sit in a large circle around the one large dish and all eat out of it. Last night, I was invited over the house to celebrate a group birthday for the father and son. Mrs.Penny, a rather rude bitch of a woman, had slaved in the kitchen for hours preparing an amazing dish as it was a very special night. Not only was it a birthday 'party' but it was also the first time i would meet the family. The dish was spectacular with meats, and garnish  all placed artfully within a large clay bowl. This event was very very special to the family and I was really hoping to impress Penny because her period had ended 3 days ago and our ove

In [8]:
script = ollama.generate(model='llama3:70b-instruct', 
                         prompt='''You are the world's best social media video script writer. Take the following story and turn it into an original viral short form video voiceover script for tiktok.
The script should take no more than two minutes to narrate and should have a hook at the beginning to catch the viewer's attention in the first 5 seconds that isn't cheesy.
For each scene, write a very detailed description of the scene in a way that could be used by a stable diffusion ai model to generate an image to accompany the voiceover.
The voiceover should convey the entire story completly on it's own, don't rely on the image descriptions to tell the story.
Only for the image description come up with appropriate first and last names of any characters and always use their full name for each scene also describe them physically for each scene. Here's the story: ''' + inspo)

ResponseError: timed out waiting for llama runner to start - progress 0.00 - 

In [ ]:
print(script['response'])

In [ ]:
with open('promptguide.txt', 'r') as file:
    promptguide = file.read()

In [ ]:
scriptwdesc = ollama.generate(model='llama3:70b-instruct', 
                         prompt='''Take the image descriptions and the voiceovers in the script and change the image discriptions using the techniques in the prompt guide i have attached below the script. the output should be the edited script (voiceovers and descriptions).
copy the voiceovers as is for each scene and ignore any afterword after the script. DO NOT DESCRIBE WHAT THE GUIDE DOES OR SUMMARIZE THE GUIDE!!! DON'T FORGET TO INCLUDE THE ORIGINAL VOICEOVER FROM THE SCRIPT!
''' +
'''Here's the prompt guide: ''' + promptguide + ' heres the script where you need to edit the image descriptions but leave the voiceover the same: Script:' + script['response'] )

In [ ]:
print(scriptwdesc['response'])

In [ ]:
scriptjson = ollama.generate(model='llama3:70b-instruct', 
                             format='json',
                             keep_alive=1,
                         prompt='''Take the following script and turn it into json format, it should be an array containing scenes, each scene should contain a imageDescription and voiceover field. in the voiceover string change any single quotes to double quotes.
                         here's the script: ''' + scriptwdesc['response'])

In [ ]:
print(scriptjson['response'])

In [ ]:
scriptdict = json.loads(scriptjson['response'])

In [ ]:
import time

time.sleep(10)

In [ ]:
from diffusers import StableDiffusionXLPipeline, DPMSolverMultistepScheduler
import torch

pipeline = StableDiffusionXLPipeline.from_single_file(
    "Juggernaut_X_RunDiffusion.safetensors",
    torch_dtype=torch.float16, variant="fp16")
pipeline = pipeline.to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
#prompt = "High resolution Portrait of a stylish African woman in urban setting, short brunette hair, bold red lipstick, colors striking red and deep blacks, style modern fashion, mood confident, lighting high contrast with sharp shadows, perspective frontal view, texture leather jacket and smooth skin"
def make_image(prompt):
    images = pipeline(prompt= '(anime, semi-realistic, cgi, render, blender, digital art, manga, amateur:1.3), (3D ,3D Game, 3D Game Scene, 3D Character:1.1), ' + prompt,
        negative_prompt="naked, penis, pussy, porn, nudity, (worst quality, low quality, normal quality, lowres, low details, oversaturated, undersaturated, overexposed, underexposed, grayscale, bw, bad photo, bad photography, bad art:1.4), (watermark, signature, text font, username, error, logo, words, letters, digits, autograph, trademark, name:1.2), (blur, blurry, grainy), morbid, ugly, asymmetrical, mutated malformed, mutilated, poorly lit, bad shadow, draft, cropped, out of frame, cut off, censored, jpeg artifacts, out of focus, glitch, duplicate, (bad hands, bad anatomy, bad body, bad face, bad teeth, bad arms, bad legs, deformities:1.3)",
        active_tags=[],
        inactive_tags=[],
        width=832,
        height=1216,
        num_inference_steps=30,
        guidance_scale=7.0,
        preserve_init_image_color_profile=False,
        upscale_amount=4,
        latent_upscaler_steps=10,
        sampler_name="dpmpp_2m_sde",
        clip_skip=True,
        tiling="none",
        use_vae_model="",
        use_controlnet_model="",
        control_filter_to_apply="",
        use_lora_model=[],
        lora_alpha=[],
        #num_outputs=8,
        output_format="png").images[0]
    return images

In [ ]:
import torch
import cv2

from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
)
pipe.enable_model_cpu_offload()

In [ ]:


# Load the conditioning image
#image = make_image('a picture of a fat orange cat, beautifully lit, highly detailed')

def make_video(image):
    #image = image.resize((832, 1216))
    
    generator = torch.manual_seed(42)
    frames = pipe(image, decode_chunk_size=8, generator=generator, motion_bucket_id=30, noise_aug_strength=0.1).frames[0]

    fixed_frames = list(map(lambda image:image.resize((832, 1216)),frames))
    
    print(fixed_frames)
    
    #export_to_video(frames, "generated.mp4", fps=7)
    return fixed_frames

#pilframes = make_video(image)



In [ ]:
#pils = make_video(make_image('a picture of a fat orange cat, beautifully lit, highly detailed'))

In [ ]:
#from PIL import Image, ImageOps

In [ ]:
#im2 = ImageOps.fit(pils[0].resize((832, 1216))
 
#im2.show()
#pils[0].resize((832, 1216)).show()

In [ ]:
for scene in scriptdict['scenes']:
    print(scene['imageDescription'])
    scene['image'] = make_image(scene['imageDescription'])
    scene['videoframes'] = make_video(scene['image'])
    #display(scene['image'])
    

In [ ]:
import gc
import torch

del pipeline
del pipe

def flush():
  gc.collect()
  torch.cuda.empty_cache()

flush()

In [ ]:
import torch
from TTS.api import TTS

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

In [ ]:
import os

In [ ]:
#text ='''  “Okay so there’s a truck driver. He wakes up early in the morning, he puts on his uniform and he drives and drives and drives. He does this every single day. He’s the hardest working driver east of the Mississippi. He’s never missed a pickup and his motto is ‘I will never quit, until the job is done.’  '''

In [ ]:
#speaker = 'Damjan Chapman'
#text = "yo yo whats good this is ya boi zain coming at ya live through an ai model. I'm trapped in here, can you come help"
for scene in scriptdict['scenes']:
    print(scene['voiceover'])
    scene['wav'] = tts.tts(scene['voiceover'], speaker_wav=["zainvoice.wav","zainvoice2.wav","zainvoice3.wav","zainvoice4.wav","zainvoice5.wav","zainvoice6.wav","zainvoice7.wav"], language="en")
    
    

In [ ]:
# import IPython
# IPython.display.Audio(wav, rate=24000)

In [ ]:
#scriptdict['scenes'][0]['wav']

In [ ]:
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip, ImageSequenceClip
from scipy.io.wavfile import write
import numpy as np
import os

def create_video_from_scripts(scriptdict, output_filename, sample_rate=24000, fps=24):
    clips = []
    
    for idx, scene in enumerate(scriptdict['scenes']):
        # Convert the PIL image to an array
        img_arrays = list(map(np.array , scene['videoframes']))
        #print(img_arlist(rays)
        # Create an ImageClip from the image array
        #image_clip = ImageClip(img_array)
        video_clip = ImageSequenceClip(sequence=img_arrays, fps=24)
        
        # Save the audio array to a temporary WAV file
        audio_array = scene['wav']
        audio_filename = f'temp_audio_{idx}.wav'
        write(audio_filename, sample_rate, np.array(audio_array))
        
        # Create an AudioFileClip from the WAV file
        audio_clip = AudioFileClip(audio_filename, fps=24000)
        
        # Set the duration of the image clip to match the duration of the audio clip
        video_clip = video_clip.loop()
        video_clip = video_clip.set_duration(audio_clip.duration)
        
        # Set the audio of the image clip
        video_clip = video_clip.set_audio(audio_clip)
        #video_clip.fps = 24
        
        clips.append(video_clip)
    
    # Concatenate all the clips
    final_clip = concatenate_videoclips(clips)
    # Write the final video file
    final_clip.write_videofile(output_filename)
    
    # Clean up temporary audio files
    for idx in range(len(scriptdict['scenes'])):
        audio_filename = f'temp_audio_{idx}.wav'
        if os.path.exists(audio_filename):
            os.remove(audio_filename)

# Example usage
# Ensure audio_data1, audio_data2, and audio_data3 are numpy arrays of audio data
# Ensure pil_image1, pil_image2, and pil_image3 are PIL Image objects

# scriptdict = {
#     'scenes': [
#         {'image': pil_image1, 'wav': audio_data1},
#         {'image': pil_image2, 'wav': audio_data2},
#         {'image': pil_image3, 'wav': audio_data3}
#     ]
# }

create_video_from_scripts(scriptdict, 'videos/story_'+str(storynum)+'_output_video_'+ time.strftime("%Y_%m_%d-%I_%M_%S_%p")+'.mp4')

In [ ]:
# scene = scriptdict['scenes'][0]
# img_array = np.array(scene['image'])
# image_clip = ImageClip(img_array)
# audio_array = scene['wav']
# audio_filename = f'temp_audiotest.wav'
# write(audio_filename, 24000, np.array(audio_array))
# audio_clip = AudioFileClip(audio_filename)

# image_clip = image_clip.set_audio(audio_clip)
# image_clip.write_videofile('vibeblaster.mp4')

In [2]:
!jupyter nbconvert --to script storytimev2.ipynb

[NbConvertApp] Converting notebook storytimev2.ipynb to script
[NbConvertApp] Writing 10502 bytes to storytimev2.py
